In [5]:
#import potrzebnych baz
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

#Połączenie z bazą danych
username = 'nick'
password = 'haslo'
host = 'localhost'
database = 'airlines'
port = port

#zmienne url oraz engine
url = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = sqlalchemy.create_engine(url)



#Funkcja modyfikująca ramke danych
def load_raw_data(file_name):
    raw_data = pd.read_csv(file_name,  encoding='utf-8')
    # Modyfikacja nazw kolumn
    raw_data.columns = [col.lower() for col in raw_data.columns]
    return raw_data

# Wczytanie airport
file_name = r"../data/raw/airport_list.csv"  
df_airport = load_raw_data(file_name)

# Wczytanie airport_weather
file_name = r"../data/raw/airport_weather.csv"  
df_airport_weather = load_raw_data(file_name)

# Wczytanie aircraft
file_name = r"../data/raw/aircraft.csv"  
df_aircraft = load_raw_data(file_name)

# Wczytanie flight
file_name = r"../data/raw/flight.csv"  
df_flight = load_raw_data(file_name)



#Dodawanie danych do tabeli
def export_table_to_db(df, table_name):
    # Sprawdzenie, czy tabela jest pusta
    table_empty = pd.read_sql(f"SELECT COUNT(*) FROM {table_name}", engine).iloc[0, 0] == 0

    # Usuwanie danych z tabeli jeśli nie jest pusta
    if not table_empty:
        engine.execute(f"DELETE FROM {table_name}")

    # Wyświetlenie informacji o ładowaniu danych
    print(f"Loading data into {table_name}...")

    # Eksport ramki danych do bazy danych
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False, chunksize=1000)
    
    

#Wgrywanie danych do tabeli
export_table_to_db(df_aircraft, 'aircraft')
export_table_to_db(df_airport_weather, 'airport_weather')
export_table_to_db(df_flight, 'flight')
export_table_to_db(df_airport, 'airport_list')



#Sprawdzenie
aircraft_expected_count = 7383
aircraft_expected_schema = ['id', 'manufacture_year', 'tail_num', 'number_of_seats']

test_data_export('aircraft', aircraft_expected_count, aircraft_expected_schema)

airport_weather_expected_count = 46226
airport_weather_expected_schema = [
       'id', 'station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tavg', 
       'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5', 'wt01', 'wt08', 'wt02',
       'wt03', 'wt04', 'wt09', 'wt06', 'wt05', 'pgtm', 'wt10', 'wesd', 'sn32',
       'sx32', 'psun', 'tsun', 'tobs', 'wt07', 'wt11', 'wt18']

test_data_export('airport_weather', airport_weather_expected_count, airport_weather_expected_schema)

flight_expected_count = 1386120
flight_expected_schema = [
       'id', 'month', 'day_of_month', 'day_of_week', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id',
       'crs_dep_time', 'dep_time', 'dep_delay_new', 'dep_time_blk',
       'crs_arr_time', 'arr_time', 'arr_delay_new', 'arr_time_blk',
       'cancelled', 'crs_elapsed_time', 'actual_elapsed_time', 'distance',
       'distance_group', 'year', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']

test_data_export('flight', flight_expected_count, flight_expected_schema)

airport_list_expected_count = 97
airport_list_expected_schema = ['id', 'origin_airport_id', 'display_airport_name', 'origin_city_name', 'name']

test_data_export('airport_list', airport_list_expected_count, airport_list_expected_schema)
